# SQL WINDOWS FUNCTIONS

Originally created by [@eugenia-anello](https://towardsdatascience.com/mastering-sql-window-functions/) and published on [Towards Data Science](https://towardsdatascience.com/). Now adapted to use duckdb as the database engine.

## Pre-requisites:

In order to practice SQL windows functions, we will need a sample database, some tables along with data to work with. We will use the `duckdb` database engine to create a sample database and populate it with data.

@eugenia-anello used a quick and smart trick to create synthetic data. See the [initial post](https://towardsdatascience.com/an-introduction-to-ctes-in-sql-ab0a979578f9/) where ChatGPT was used to define a database and generate synthetic data. 

We will use similar approach to create a sample database and populate it with data. Instead of sales data, let use PEPFAR/HIV data. We will modify the prompt and use Gemini instead of ChatGPT to generate the SQL code to create the database and populate it with data.

Here is the original prompt used for `ChatGPT`:

```
I want to create a database about sales of a fashion online company, 
zalando: create tables and insert rows using SQL. 
The goal of this database is to demonstrate the strenghts of SQL CTE. 
It should contain syntetic data that resemble real data between 2023 and 2024.
``` 

Here is the modified prompt used for `Gemini (2.5 Flash)`:

```
I want to create a database about PEPFAR/HIV treatment programs for an NGO,
AfricaAID: create tables and insert rows using SQL.
The goal of this database is to demonstrate the strenghts of SQL CTE.
It should contain synthetic data that resemble real data between 2023 and 2024.
```

The output of the prompt is located at `./pyanalytics/queries/gemini-africa-aid-hiv-database.sql`.

## Implementation

We will be using `duckdb` as the database engine to implement a sample database and populate it with data.

In [156]:
# Libraries
import duckdb
import pandas as pd

In [157]:
duckdb.sql("SELECT version() AS version").show()

┌─────────┐
│ version │
│ varchar │
├─────────┤
│ v1.3.1  │
└─────────┘



In [158]:
# Test DuckDB Engine
duckdb.sql("SELECT DATE('2003-05-27') AS pepfar_start_date").show()

┌───────────────────┐
│ pepfar_start_date │
│       date        │
├───────────────────┤
│ 2003-05-27        │
└───────────────────┘



In case you didn't know, PEPFAR (the U.S. President's Emergency Plan for AIDS Relief) is a U.S. government initiative to address the global HIV/AIDS epidemic and help save the lives of those suffering from the disease, particularly in Africa. The initiative was launched in 2003 and has since provided billions of dollars in funding for HIV/AIDS prevention, treatment, and care programs in more than 50 countries.
More info can be found at [PEPFAR](https://www.pepfar.gov/) or [HHS/HRSA](https://www.hrsa.gov/office-global-health/global-hivaids-program/about-pepfar).

### Dababase Setup

In [159]:
from dotenv import load_dotenv
import os
from pathlib import Path

In [ ]:
# Load environment variables from .env file
load_dotenv()

In [150]:
# Python notebook uses the parent directory of the current file as its working directory
# so we need to be specific about the project root directory

#print(os.getenv("DATA_DIR"))
#(os.path.exists(os.getenv("DATA_DIR")))

#print(os.getcwd())
#print(Path(os.getcwd()).parent.parent)
#print(os.listdir(Path(os.getcwd()).parent.parent))

In [161]:
# Data directory
# This directory is used to store data files and should be set in the .env file
# or default to "data" if not specified.

# Make sure that the directory exists in the root.
dir_data = Path(os.getcwd()).parent.parent / "data"

if not os.path.exists(dir_data):
    os.makedirs(dir_data)

# Directory for data files
DIR_DATA = os.getenv("DATA_DIR", dir_data)


In [166]:
print(f"Data directory: {DIR_DATA}")

Data directory: ~/Documents/Projects/data


In [167]:
# Let Create a DuckDB Database in the data directory

db_name = "AfricaAID.db"

db_path = Path(DIR_DATA) / db_name

In [168]:
print(db_path)

~/Documents/Projects/data/AfricaAID.db


In [169]:
conn = duckdb.connect(str(db_path))

### Database tables definitions

In [170]:
# Table - Countries

# Create a duckdb table primary key sequence for Countries/country_id
conn.execute("CREATE SEQUENCE IF NOT EXISTS country_id_seq START WITH 1 INCREMENT BY 1")

# Create countries table with country_id as primary key and automatic increment
conn.execute(
    """CREATE TABLE Countries (
        country_id INT PRIMARY KEY DEFAULT nextval('country_id_seq'),
        country_name VARCHAR(100) NOT NULL,
        continent VARCHAR(50) NOT NULL
    )"""
)

In [171]:
# Table - Implementing Partners
# With partner types: e.g., 'Local NGO', 'International NGO', 'Government Agency'
conn.execute("CREATE SEQUENCE IF NOT EXISTS partner_id_seq START WITH 1 INCREMENT BY 1")

conn.execute(
    """CREATE TABLE Partners (
        partner_id INT PRIMARY KEY DEFAULT nextval('partner_id_seq'),
        partner_name VARCHAR(255) NOT NULL,
        partner_type VARCHAR(100)
    )"""
)

In [172]:
# Table - Programs Sites
# With site types: e.g., 'Hospital', 'Clinic', 'Community Center'

conn.execute("CREATE SEQUENCE IF NOT EXISTS site_id_seq START WITH 1 INCREMENT BY 1")

conn.execute(
    """CREATE TABLE ProgramSites (
        site_id INT PRIMARY KEY DEFAULT nextval('site_id_seq'),
        site_name VARCHAR(255) NOT NULL,
        country_id INT,
        partner_id INT,
        site_type VARCHAR(100),
        latitude DECIMAL(9, 6),
        longitude DECIMAL(9, 6),
        FOREIGN KEY (country_id) REFERENCES Countries(country_id),
        FOREIGN KEY (partner_id) REFERENCES Partners(partner_id)
    )"""
)

In [173]:
# Table - Treatment Enrollment
## With patient gender options -- 'Male', 'Female', 'Other'
## With -- e.g., 'Active', 'Lost to Follow-up', 'Deceased'

conn.execute("CREATE SEQUENCE IF NOT EXISTS enrollment_id_seq START WITH 1 INCREMENT BY 1")

conn.execute(
    """CREATE TABLE TreatmentEnrollments (
        enrollment_id INT PRIMARY KEY DEFAULT nextval('enrollment_id_seq'),
        site_id INT,
        enrollment_date DATE NOT NULL,
        patient_age INT,
        patient_gender VARCHAR(10),
        treatment_status VARCHAR(50),
        FOREIGN KEY (site_id) REFERENCES ProgramSites(site_id)
    )"""
)

In [174]:
# Table - Funding Allocations
# With funding category -- e.g., 'Prevention', 'Treatment', 'Care & Support', 'Lab Services'

conn.execute("CREATE SEQUENCE IF NOT EXISTS funding_id_seq START WITH 1 INCREMENT BY 1")

conn.execute(
    """CREATE TABLE FundingAllocations (
        funding_id INT PRIMARY KEY DEFAULT nextval('funding_id_seq'),
        partner_id INT,
        country_id INT,
        fiscal_year INT NOT NULL,
        allocated_amount DECIMAL(18, 2) NOT NULL,
        fund_category VARCHAR(100),
        FOREIGN KEY (partner_id) REFERENCES Partners(partner_id),
        FOREIGN KEY (country_id) REFERENCES Countries(country_id)
    )"""
)

### Database - Data Insertions

In [175]:
# Table - Countries

conn.execute(
    """INSERT INTO Countries (country_name, continent) VALUES
        ('South Africa', 'Africa'),
        ('Kenya', 'Africa'),
        ('Uganda', 'Africa'),
        ('Tanzania', 'Africa'),
        ('Zambia', 'Africa')
    """
)

In [176]:
# Table - Partners

conn.execute(
    """INSERT INTO Partners (partner_name, partner_type) VALUES
        ('Global Health Initiative', 'International NGO'),
        ('African AIDS Network', 'Local NGO'),
        ('Ministry of Health - Kenya', 'Government Agency'),
        ('Youth Empowerment Foundation', 'Local NGO'),
        ('Disease Control Alliance', 'International NGO')
    """
)

In [177]:
# Table - Program Sites

conn.execute(
    """INSERT INTO ProgramSites (site_name, country_id, partner_id, site_type, latitude, longitude) VALUES
        ('Pretoria General Hospital', 1, 1, 'Hospital', -25.747868, 28.229271),
        ('Nairobi Community Clinic', 2, 3, 'Clinic', -1.292066, 36.821946),
        ('Kampala Central Health Center', 3, 2, 'Community Center', 0.347596, 32.582520),
        ('Dar es Salaam Urban Clinic', 4, 1, 'Clinic', -6.792354, 39.208328),
        ('Lusaka Regional Hospital', 5, 5, 'Hospital', -15.387520, 28.322817),
        ('Cape Town HIV Wellness Center', 1, 2, 'Community Center', -33.924900, 18.424100),
        ('Kisumu Referral Hospital', 2, 1, 'Hospital', -0.091700, 34.767900),
        ('Mbarara District Clinic', 3, 4, 'Clinic', -0.604800, 30.655800),
        ('Dodoma Regional Clinic', 4, 3, 'Clinic', -6.163100, 35.751900),
        ('Ndola Urban Health Post', 5, 2, 'Community Center', -12.981800, 28.636600)
    """
)

In [178]:
# Table - Treatment Enrollments

conn.execute(
    """INSERT INTO TreatmentEnrollments (site_id, enrollment_date, patient_age, patient_gender, treatment_status) VALUES
        (1, '2023-01-15', 35, 'Female', 'Active'),
        (2, '2023-02-01', 28, 'Male', 'Active'),
        (3, '2023-03-10', 42, 'Female', 'Active'),
        (4, '2023-04-05', 19, 'Male', 'Active'),
        (5, '2023-05-20', 50, 'Female', 'Active'),
        (1, '2023-06-01', 22, 'Male', 'Active'),
        (2, '2023-07-11', 30, 'Female', 'Active'),
        (3, '2023-08-19', 45, 'Male', 'Lost to Follow-up'),
        (4, '2023-09-02', 26, 'Female', 'Active'),
        (5, '2023-10-14', 38, 'Male', 'Active'),
        (6, '2023-11-20', 31, 'Female', 'Active'),
        (7, '2023-12-05', 24, 'Male', 'Active'),
        (8, '2023-12-25', 55, 'Female', 'Active'),
        (1, '2024-01-08', 29, 'Female', 'Active'),
        (2, '2024-02-18', 33, 'Male', 'Active'),
        (3, '2024-03-01', 40, 'Female', 'Active'),
        (4, '2024-04-12', 21, 'Male', 'Active'),
        (5, '2024-05-01', 48, 'Female', 'Active'),
        (6, '2024-06-10', 27, 'Male', 'Active'),
        (7, '2024-07-01', 36, 'Female', 'Active'),
        (8, '2024-08-15', 52, 'Male', 'Deceased'),
        (9, '2024-09-01', 23, 'Female', 'Active'),
        (10, '2024-10-20', 39, 'Male', 'Active'),
        (1, '2024-11-05', 30, 'Female', 'Active'),
        (2, '2024-12-10', 25, 'Male', 'Active')
    """
)

In [179]:
# Table - Funding Allocations

conn.execute(
    """INSERT INTO FundingAllocations (partner_id, country_id, fiscal_year, allocated_amount, fund_category) VALUES
        (1, 1, 2023, 500000.00, 'Treatment'),
        (3, 2, 2023, 300000.00, 'Prevention'),
        (2, 3, 2023, 250000.00, 'Care & Support'),
        (1, 4, 2023, 400000.00, 'Treatment'),
        (5, 5, 2023, 350000.00, 'Lab Services'),
        (2, 1, 2023, 150000.00, 'Prevention'),
        (1, 2, 2023, 200000.00, 'Care & Support'),
        (1, 1, 2024, 550000.00, 'Treatment'),
        (3, 2, 2024, 320000.00, 'Prevention'),
        (2, 3, 2024, 270000.00, 'Care & Support'),
        (1, 4, 2024, 420000.00, 'Treatment'),
        (5, 5, 2024, 380000.00, 'Lab Services'),
        (4, 3, 2024, 100000.00, 'Prevention'),
        (3, 4, 2024, 180000.00, 'Care & Support')
    """
)

## Database Queries

In [180]:
# List of table (with details) within the DuckDB database
conn.sql("SHOW ALL TABLES").show()

┌───────────┬─────────┬──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────┬───────────┐
│ database  │ schema  │         name         │                                       column_names                                       │                                 column_types                                  │ temporary │
│  varchar  │ varchar │       varchar        │                                        varchar[]                                         │                                   varchar[]                                   │  boolean  │
├───────────┼─────────┼──────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────┼───────────┤
│ AfricaAID │ main    │ Countries            │ [country_id, country_name, contin

In [181]:
# Simple list of tables name
conn.sql("SHOW TABLES").show()

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ Countries            │
│ FundingAllocations   │
│ Partners             │
│ ProgramSites         │
│ TreatmentEnrollments │
└──────────────────────┘



In [182]:
# Table Structure - Countries
conn.sql("SHOW Countries").show()

┌──────────────┬─────────────┬─────────┬─────────┬───────────────────────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │          default          │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │          varchar          │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼───────────────────────────┼─────────┤
│ country_id   │ INTEGER     │ NO      │ PRI     │ nextval('country_id_seq') │ NULL    │
│ country_name │ VARCHAR     │ NO      │ NULL    │ NULL                      │ NULL    │
│ continent    │ VARCHAR     │ NO      │ NULL    │ NULL                      │ NULL    │
└──────────────┴─────────────┴─────────┴─────────┴───────────────────────────┴─────────┘



In [183]:
# Table Structure - Partners
conn.sql("DESCRIBE Partners").show()

┌──────────────┬─────────────┬─────────┬─────────┬───────────────────────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │          default          │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │          varchar          │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼───────────────────────────┼─────────┤
│ partner_id   │ INTEGER     │ NO      │ PRI     │ nextval('partner_id_seq') │ NULL    │
│ partner_name │ VARCHAR     │ NO      │ NULL    │ NULL                      │ NULL    │
│ partner_type │ VARCHAR     │ YES     │ NULL    │ NULL                      │ NULL    │
└──────────────┴─────────────┴─────────┴─────────┴───────────────────────────┴─────────┘



In [185]:
# Tables Content - Countries
conn.sql("SELECT * FROM Countries").show()

┌────────────┬──────────────┬───────────┐
│ country_id │ country_name │ continent │
│   int32    │   varchar    │  varchar  │
├────────────┼──────────────┼───────────┤
│          1 │ South Africa │ Africa    │
│          2 │ Kenya        │ Africa    │
│          3 │ Uganda       │ Africa    │
│          4 │ Tanzania     │ Africa    │
│          5 │ Zambia       │ Africa    │
└────────────┴──────────────┴───────────┘



In [186]:
# # of countries by continent
conn.sql(
    """SELECT continent, COUNT (Country_name) as n_countries
        FROM Countries
        GROUP BY continent"""
    ).show()

┌───────────┬─────────────┐
│ continent │ n_countries │
│  varchar  │    int64    │
├───────────┼─────────────┤
│ Africa    │           5 │
└───────────┴─────────────┘



In [187]:
# Tables Content - Partners
conn.sql("SELECT * FROM Partners").show()

┌────────────┬──────────────────────────────┬───────────────────┐
│ partner_id │         partner_name         │   partner_type    │
│   int32    │           varchar            │      varchar      │
├────────────┼──────────────────────────────┼───────────────────┤
│          1 │ Global Health Initiative     │ International NGO │
│          2 │ African AIDS Network         │ Local NGO         │
│          3 │ Ministry of Health - Kenya   │ Government Agency │
│          4 │ Youth Empowerment Foundation │ Local NGO         │
│          5 │ Disease Control Alliance     │ International NGO │
└────────────┴──────────────────────────────┴───────────────────┘



In [188]:
# # of partners by type
conn.sql(
    """SELECT partner_type, COUNT (partner_id) as n_partners
        FROM Partners
        GROUP BY partner_type
        ORDER BY n_partners DESC"""
    ).show()

┌───────────────────┬────────────┐
│   partner_type    │ n_partners │
│      varchar      │   int64    │
├───────────────────┼────────────┤
│ International NGO │          2 │
│ Local NGO         │          2 │
│ Government Agency │          1 │
└───────────────────┴────────────┘



In [189]:
# Tables Content - Sites
conn.sql("SELECT * FROM ProgramSites").show()

┌─────────┬───────────────────────────────┬────────────┬────────────┬──────────────────┬──────────────┬──────────────┐
│ site_id │           site_name           │ country_id │ partner_id │    site_type     │   latitude   │  longitude   │
│  int32  │            varchar            │   int32    │   int32    │     varchar      │ decimal(9,6) │ decimal(9,6) │
├─────────┼───────────────────────────────┼────────────┼────────────┼──────────────────┼──────────────┼──────────────┤
│       1 │ Pretoria General Hospital     │          1 │          1 │ Hospital         │   -25.747868 │    28.229271 │
│       2 │ Nairobi Community Clinic      │          2 │          3 │ Clinic           │    -1.292066 │    36.821946 │
│       3 │ Kampala Central Health Center │          3 │          2 │ Community Center │     0.347596 │    32.582520 │
│       4 │ Dar es Salaam Urban Clinic    │          4 │          1 │ Clinic           │    -6.792354 │    39.208328 │
│       5 │ Lusaka Regional Hospital      │     

In [190]:
# # of program sites by country and partner
conn.sql(
    """SELECT c.country_name, COUNT(s.site_id) AS n_sites
        FROM ProgramSites s
        JOIN Countries c ON s.country_id = c.country_id
        GROUP BY c.country_name
        ORDER BY n_sites DESC, c.country_name ASC"""
).show()

┌──────────────┬─────────┐
│ country_name │ n_sites │
│   varchar    │  int64  │
├──────────────┼─────────┤
│ Kenya        │       2 │
│ South Africa │       2 │
│ Tanzania     │       2 │
│ Uganda       │       2 │
│ Zambia       │       2 │
└──────────────┴─────────┘



In [191]:
# Tables Content - TX Enrollments
conn.sql("SELECT * FROM TreatmentEnrollments").show()

┌───────────────┬─────────┬─────────────────┬─────────────┬────────────────┬───────────────────┐
│ enrollment_id │ site_id │ enrollment_date │ patient_age │ patient_gender │ treatment_status  │
│     int32     │  int32  │      date       │    int32    │    varchar     │      varchar      │
├───────────────┼─────────┼─────────────────┼─────────────┼────────────────┼───────────────────┤
│             1 │       1 │ 2023-01-15      │          35 │ Female         │ Active            │
│             2 │       2 │ 2023-02-01      │          28 │ Male           │ Active            │
│             3 │       3 │ 2023-03-10      │          42 │ Female         │ Active            │
│             4 │       4 │ 2023-04-05      │          19 │ Male           │ Active            │
│             5 │       5 │ 2023-05-20      │          50 │ Female         │ Active            │
│             6 │       1 │ 2023-06-01      │          22 │ Male           │ Active            │
│             7 │       2 │ 20

In [ ]:
# # of enrollments by treatment status - using a simple window function to calculate percentages
conn.sql(
    """SELECT
        e.treatment_status as status,
        COUNT(e.enrollment_id) as n_patients,
        ROUND(COUNT(e.enrollment_id) / SUM(COUNT(e.enrollment_id)) OVER (), 2) AS pct_patients,
        FROM TreatmentEnrollments e
        JOIN ProgramSites s ON e.site_id = s.site_id
        GROUP BY e.treatment_status,
        ORDER BY e.treatment_status, n_patients DESC"""
).show()

┌───────────────────┬────────────┬──────────────┐
│      status       │ n_patients │ pct_patients │
│      varchar      │   int64    │    double    │
├───────────────────┼────────────┼──────────────┤
│ Active            │         23 │         0.92 │
│ Deceased          │          1 │         0.04 │
│ Lost to Follow-up │          1 │         0.04 │
└───────────────────┴────────────┴──────────────┘



In [ ]:
# # of enrollments by country and gender - Using a Partitioned Window Function
conn.sql(
    """SELECT
        c.country_name as country,
        e.patient_gender as gender,
        COUNT(e.enrollment_id) as n_patients,
        ROUND(COUNT(e.enrollment_id) / SUM(COUNT(e.enrollment_id)) OVER (PARTITION BY c.country_name), 2) AS pct_patients,
        FROM TreatmentEnrollments e
        JOIN ProgramSites s ON e.site_id = s.site_id
        JOIN Countries c ON s.country_id = c.country_id
        GROUP BY c.country_name, e.patient_gender
        ORDER BY c.country_name, n_patients DESC"""
).show()

┌──────────────┬─────────┬────────────┬──────────────┐
│   country    │ gender  │ n_patients │ pct_patients │
│   varchar    │ varchar │   int64    │    double    │
├──────────────┼─────────┼────────────┼──────────────┤
│ Kenya        │ Male    │          4 │         0.67 │
│ Kenya        │ Female  │          2 │         0.33 │
│ South Africa │ Female  │          4 │         0.67 │
│ South Africa │ Male    │          2 │         0.33 │
│ Tanzania     │ Female  │          2 │          0.5 │
│ Tanzania     │ Male    │          2 │          0.5 │
│ Uganda       │ Female  │          3 │          0.6 │
│ Uganda       │ Male    │          2 │          0.4 │
│ Zambia       │ Female  │          2 │          0.5 │
│ Zambia       │ Male    │          2 │          0.5 │
├──────────────┴─────────┴────────────┴──────────────┤
│ 10 rows                                  4 columns │
└────────────────────────────────────────────────────┘



In [210]:
# # of enrollments by country and gender - Using a Partitioned Window Function
conn.sql(
    """SELECT
        c.country_name as country,
        YEAR(e.enrollment_date) as enrollment_year,
        COUNT(e.enrollment_id) as n_patients,
        ROUND(COUNT(e.enrollment_id) / SUM(COUNT(e.enrollment_id)) OVER (PARTITION BY c.country_name, YEAR(e.enrollment_date)), 2) AS pct_patients,
        FROM TreatmentEnrollments e
        JOIN ProgramSites s ON e.site_id = s.site_id
        JOIN Countries c ON s.country_id = c.country_id
        GROUP BY c.country_name, YEAR(e.enrollment_date)
        ORDER BY c.country_name, n_patients DESC"""
).show()

┌──────────────┬─────────────────┬────────────┬──────────────┐
│   country    │ enrollment_year │ n_patients │ pct_patients │
│   varchar    │      int64      │   int64    │    double    │
├──────────────┼─────────────────┼────────────┼──────────────┤
│ Kenya        │            2024 │          3 │          1.0 │
│ Kenya        │            2023 │          3 │          1.0 │
│ South Africa │            2023 │          3 │          1.0 │
│ South Africa │            2024 │          3 │          1.0 │
│ Tanzania     │            2023 │          2 │          1.0 │
│ Tanzania     │            2024 │          2 │          1.0 │
│ Uganda       │            2023 │          3 │          1.0 │
│ Uganda       │            2024 │          2 │          1.0 │
│ Zambia       │            2024 │          2 │          1.0 │
│ Zambia       │            2023 │          2 │          1.0 │
├──────────────┴─────────────────┴────────────┴──────────────┤
│ 10 rows                                          4 co

In [ ]:
# SIMPLE CTE -  Enrollment Proportions by Country and Treatment Status

conn.sql(
    """
    WITH EnrollmentStatus AS (
        SELECT c.country_name as country,
        e.treatment_status as status,
        COUNT(e.enrollment_id) as n_patients,
        SUM(COUNT(e.enrollment_id)) OVER (PARTITION BY c.country_name) AS cntry_patients
        FROM TreatmentEnrollments e
        JOIN ProgramSites s ON e.site_id = s.site_id
        JOIN Countries c ON s.country_id = c.country_id
        GROUP BY c.country_name, e.treatment_status
    )
    SELECT
        country, status, n_patients, cntry_patients,
        ROUND(n_patients / cntry_patients * 100, 2) AS pct_patients,
    FROM EnrollmentStatus
    ORDER BY country, status, pct_patients DESC
    """
).show()

┌──────────────┬───────────────────┬────────────┬────────────────┬──────────────┐
│   country    │      status       │ n_patients │ cntry_patients │ pct_patients │
│   varchar    │      varchar      │   int64    │     int128     │    double    │
├──────────────┼───────────────────┼────────────┼────────────────┼──────────────┤
│ Kenya        │ Active            │          6 │              6 │        100.0 │
│ South Africa │ Active            │          6 │              6 │        100.0 │
│ Tanzania     │ Active            │          4 │              4 │        100.0 │
│ Uganda       │ Active            │          3 │              5 │         60.0 │
│ Uganda       │ Deceased          │          1 │              5 │         20.0 │
│ Uganda       │ Lost to Follow-up │          1 │              5 │         20.0 │
│ Zambia       │ Active            │          4 │              4 │        100.0 │
└──────────────┴───────────────────┴────────────┴────────────────┴──────────────┘



In [212]:
# CTEs - Percentage of Patients by country and by treatment status

conn.sql(
    """
    WITH
        -- CTE 1: Count enrollments per country and treatment status
        EnrollmentsByCountryAndStatus AS (
        SELECT
            c.country_name,
            te.treatment_status,
            COUNT(te.enrollment_id) AS n_enrollments
        FROM
            TreatmentEnrollments te
        JOIN
            ProgramSites ps ON te.site_id = ps.site_id
        JOIN
            Countries c ON ps.country_id = c.country_id
        GROUP BY
            c.country_name,
            te.treatment_status
    ),
    -- CTE 2: Count total enrollments per country
    TotalEnrollmentsByCountry AS (
        SELECT
            c.country_name,
            COUNT(te.enrollment_id) AS total_enrollments
        FROM
            TreatmentEnrollments te
        JOIN
            ProgramSites ps ON te.site_id = ps.site_id
        JOIN
            Countries c ON ps.country_id = c.country_id
        GROUP BY
            c.country_name
    )
    -- Main query: Calculate percentage
    SELECT
        ecas.country_name as country,
        ecas.treatment_status as status,
        ecas.n_enrollments as enrollments,
        tebc.total_enrollments,
        (CAST(ecas.n_enrollments AS REAL) * 100.0 / tebc.total_enrollments) AS prop_patients
    FROM
        EnrollmentsByCountryAndStatus ecas
    JOIN
        TotalEnrollmentsByCountry tebc ON ecas.country_name = tebc.country_name
    ORDER BY
        ecas.country_name,
        ecas.treatment_status
    """
)

┌──────────────┬───────────────────┬─────────────┬───────────────────┬───────────────┐
│   country    │      status       │ enrollments │ total_enrollments │ prop_patients │
│   varchar    │      varchar      │    int64    │       int64       │     float     │
├──────────────┼───────────────────┼─────────────┼───────────────────┼───────────────┤
│ Kenya        │ Active            │           6 │                 6 │         100.0 │
│ South Africa │ Active            │           6 │                 6 │         100.0 │
│ Tanzania     │ Active            │           4 │                 4 │         100.0 │
│ Uganda       │ Active            │           3 │                 5 │          60.0 │
│ Uganda       │ Deceased          │           1 │                 5 │          20.0 │
│ Uganda       │ Lost to Follow-up │           1 │                 5 │          20.0 │
│ Zambia       │ Active            │           4 │                 4 │         100.0 │
└──────────────┴───────────────────┴───────

In [213]:
# CTE: Calculate TX Enrollments per country and status, and
# total enrollments per country using a window function
conn.sql(
    """
    WITH
    EnrollmentsData AS (
        SELECT
            c.country_name as country,
            te.treatment_status as status,
            -- # of enrollments per group
            COUNT(te.enrollment_id) AS enrollments,
            -- Window function to calculate roll up total enrollments
            SUM(COUNT(te.enrollment_id)) OVER (PARTITION BY c.country_name) AS total_enrollments
        FROM
            TreatmentEnrollments te
        JOIN
            ProgramSites ps ON te.site_id = ps.site_id
        JOIN
            Countries c ON ps.country_id = c.country_id
        GROUP BY
            c.country_name,
            te.treatment_status
    )
    -- Main query: Calculate percentage using the results from the single CTE
    SELECT
        country, status, enrollments, total_enrollments,
        (CAST(enrollments AS REAL) * 100.0 / total_enrollments) AS prop_patients
    FROM EnrollmentsData
    ORDER BY country, status
    """
)

┌──────────────┬───────────────────┬─────────────┬───────────────────┬───────────────┐
│   country    │      status       │ enrollments │ total_enrollments │ prop_patients │
│   varchar    │      varchar      │    int64    │      int128       │     float     │
├──────────────┼───────────────────┼─────────────┼───────────────────┼───────────────┤
│ Kenya        │ Active            │           6 │                 6 │         100.0 │
│ South Africa │ Active            │           6 │                 6 │         100.0 │
│ Tanzania     │ Active            │           4 │                 4 │         100.0 │
│ Uganda       │ Active            │           3 │                 5 │          60.0 │
│ Uganda       │ Deceased          │           1 │                 5 │          20.0 │
│ Uganda       │ Lost to Follow-up │           1 │                 5 │          20.0 │
│ Zambia       │ Active            │           4 │                 4 │         100.0 │
└──────────────┴───────────────────┴───────

In [214]:
# WINDOW FUNCTION - CTE & RANKING ENROLLMENTS BY COUNTRY AND TREATMENT STATUS

conn.sql(
    """
    WITH EnrollmentCounts AS (
        SELECT
            c.country_name AS country,
            te.treatment_status AS status,
            COUNT(te.enrollment_id) AS n_enrollments
        FROM TreatmentEnrollments te
        JOIN ProgramSites ps ON te.site_id = ps.site_id
        JOIN Countries c ON ps.country_id = c.country_id
        GROUP BY c.country_name, te.treatment_status
    )
    SELECT
        country, status, n_enrollments,
        RANK() OVER (PARTITION BY country ORDER BY n_enrollments DESC) AS rank_within_country
    FROM EnrollmentCounts
    ORDER BY country, rank_within_country;
    """
).show()

┌──────────────┬───────────────────┬───────────────┬─────────────────────┐
│   country    │      status       │ n_enrollments │ rank_within_country │
│   varchar    │      varchar      │     int64     │        int64        │
├──────────────┼───────────────────┼───────────────┼─────────────────────┤
│ Kenya        │ Active            │             6 │                   1 │
│ South Africa │ Active            │             6 │                   1 │
│ Tanzania     │ Active            │             4 │                   1 │
│ Uganda       │ Active            │             3 │                   1 │
│ Uganda       │ Lost to Follow-up │             1 │                   2 │
│ Uganda       │ Deceased          │             1 │                   2 │
│ Zambia       │ Active            │             4 │                   1 │
└──────────────┴───────────────────┴───────────────┴─────────────────────┘



In [215]:
conn.close()